In [1]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_IT.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IT/functions_IT_v1.ipynb

In [28]:
it_05 = xr.open_dataset('/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/ssh_it_05.nc', engine='netcdf4')
it_06 = xr.open_dataset('/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/ssh_it_06.nc', engine='netcdf4')
it_07 = xr.open_dataset('/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/ssh_it_07.nc', engine='netcdf4')

In [29]:
it_05 #corregir todo se esta guardando en mayo

<xarray.Dataset> Size: 12GB
Dimensions:       (time_counter: 2208, y: 853, x: 788)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 18kB 2014-05-01T00:30:00 ... ...
    nav_lat       (y, x) float32 3MB ...
    nav_lon       (y, x) float32 3MB ...
Dimensions without coordinates: y, x
Data variables:
    ssh_it        (time_counter, y, x) float64 12GB ...

In [43]:
# Assuming 'it_05' is your xarray.Dataset
it_05_dataarray = it_05['ssh_it']

# Drop the coordinates 'nav_lat' and 'nav_lon'
it_05_dataarray = it_05_dataarray.drop(['nav_lat', 'nav_lon'])

# Now apply the isotropic_power_spectrum function to the DataArray
res_it = xrft.isotropic_power_spectrum(it_05_dataarray, dim=['x', 'y']).mean(["time_counter"])

# Print the result to verify
print(res_it)


/tmp/ipykernel_3058911/718544880.py:5: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  it_05_dataarray = it_05_dataarray.drop(['nav_lat', 'nav_lon'])


: 

In [39]:
import xarray as xr
import numpy as np
# Assuming these classes/functions are defined elsewhere and imported correctly
# from your_module import Axis, TemporalAxis, Grid3D, fill

def interpolate_ssh_it(ssh_it):
    # Check if the 'ssh_it' variable is in the dataset
    if 'ssh_it' not in ssh_it.data_vars:
        raise ValueError("Dataset does not contain the expected data variable 'ssh_it'")

    # Extract the sea surface height data variable
    ssh_data = ssh_it['ssh_it']

    # Handling potentially 2D latitude and longitude arrays
    if ssh_it.nav_lon.ndim > 1:
        lon1d = ssh_it.nav_lon.isel(y=0).values  # Assuming consistent longitude along the first row
        lat1d = ssh_it.nav_lat.isel(x=0).values  # Assuming consistent latitude along the first column
    else:
        lon1d = ssh_it.nav_lon.values
        lat1d = ssh_it.nav_lat.values

    x_axis = Axis(lon1d, is_circle=True)
    y_axis = Axis(lat1d, is_circle=True)
    t_axis = TemporalAxis(ssh_it.time_counter.values)

    # Ensure the dimensions order when accessing the data array
    grid = Grid3D(y_axis, x_axis, t_axis, ssh_data.transpose('time_counter', 'y', 'x').values)
    has_converged, filled = fill.gauss_seidel(grid, num_threads=4)

    ssh_data_filled = ssh_data.copy(data=filled.transpose(2, 0, 1)).chunk({'time_counter': 1})

    # Using a rough degree conversion for km to degrees
    dx_deg = 2 / 111.32  # Convert dx from km to degrees latitude
    ENSLAT2D, ENSLON2D = np.meshgrid(
        np.arange(lat1d.min(), lat1d.max(), dx_deg),
        np.arange(lon1d.min(), lon1d.max(), dx_deg),
        indexing='ij'
    )

    # Interpolating onto the Cartesian grid using scipy griddata or similar method
    array_cart_ssh = griddata(
        (lat1d.repeat(len(lon1d)), lon1d.tile(len(lat1d))),
        ssh_data_filled.values.flatten(),
        (ENSLAT2D, ENSLON2D),
        method='linear'
    )

    # Fill NaNs in the interpolated data
    x_axis = Axis(np.arange(array_cart_ssh.shape[1]))
    y_axis = Axis(np.arange(array_cart_ssh.shape[0]))
    grid = Grid3D(y_axis, x_axis, array_cart_ssh)
    has_converged, filled = fill.gauss_seidel(grid, num_threads=4)

    cart_ssh_it = xr.DataArray(
        data=filled,
        dims=["lat", "lon"],
        coords={"lat": ENSLAT2D[:, 0], "lon": ENSLON2D[0, :]}
    ).chunk({'lat': 1, 'lon': 1})

    return cart_ssh_it

# Usage:
# it_caledo = interpolate_ssh_it(it_05)


In [40]:
it_caledo = interpolate_ssh_it(it_05)

ValueError: x, array could not be broadcast together with shape (853, ) (2208, 853, 788, )